### 1.0) Iniciando o desafio

Para esse desafio, queremos criar um chatBOT que nos fornece informações em tempo real do mercado financeiro. Para tal, vamos utilizar uma biblioteca do yahoo finance que é outra API para obter esses dados.

##### 1º) importando as bibliotecas

In [1]:
import json
import yfinance as yf
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
client = openai.Client()

##### 2º) Criando a função

In [2]:
def retorna_cotacao_historica(ticker, periodo): # ticker é o nome da ação, e periodo é a data
    ticker_obj = yf.Ticker(ticker) # é o objeto da ação listada
    hist = ticker_obj.history(period = periodo, auto_adjust=False) # isso gera uma tabela com as datas da ação, mostrando o preço minimo, max, etc...
    if len(hist) > 30: # caso eu tenha mais que 30 entradas
        slice_size = int(len(hist)/30) # suponha 60 entradas, eu divido em pedaços de 2
        hist = hist.iloc[::-slice_size][::-1] ## é um comando para encurtar as entradas dividindo pelo tamanho desejado. Basicamente faz ele pular os dados de acordo com o slice_size
    hist.index = hist.index.strftime('%m-%d-%Y') # converte a data para esse formato
    return hist['Close'].to_json() #retorna a data de fechamento


##### 3º) Criando o Tools


In [3]:
tools = [
    {
        'type':'function',
        'function':{
            'name':'retorna_cotacao_historica',
            'description': 'Retorna a cotação de uma determinada ação em um período definido',
            'parameters':{
                'type':'object',
                'properties':{
                    'ticker':{
                        'type':'string',
                        'description':'É a ação que queremos obter informações'
                    },
                    'periodo':{
                        'type':'string',
                        'description':'O período desejado',
                        'enum':['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
                    },
                },

            },
        },
    }
]

##### 4º) Criando o script da pergunta


In [4]:

funcoes_disponiveis = {
    'retorna_cotacao_historica': retorna_cotacao_historica
}


mensagens = [
    {'role':'user', 'content':'Qual é a cotação da ambev atual e qual é sua variação de preço em 1 ano?'}
]


resposta = client.chat.completions.create(
    model = 'gpt-3.5-turbo-0125',
    messages = mensagens,
    tools = tools,
    tool_choice = 'auto')

In [5]:
resposta

ChatCompletion(id='chatcmpl-AuNmad2XWcSp3ZERuKErZZmdnH1Bt', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_P9oZt26IwxMx6QMVgw5NsSLe', function=Function(arguments='{"ticker": "ABEV3.SA", "periodo": "1d"}', name='retorna_cotacao_historica'), type='function'), ChatCompletionMessageToolCall(id='call_BpNGG0EzJ9YiS6wnJtINHz6R', function=Function(arguments='{"ticker": "ABEV3.SA", "periodo": "1y"}', name='retorna_cotacao_historica'), type='function')]))], created=1738001152, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=76, prompt_tokens=160, total_tokens=236, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_det

In [6]:
#ChatCompletionMessageToolCall(id='call_yVCl0MV3LB3GV7WdcG9n7HFw', function=Function(arguments='{"ticker": "ABEV3.SA", "periodo": "1d"}', name='retorna_cotacao_historica'), type='function')

mensagem_resp = resposta.choices[0].message # isso representa a mensagem que o chat deu, leia em resposta
tool_calls = mensagem_resp.tool_calls

if tool_calls:
    mensagens.append(mensagem_resp)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[function_name]
        function_arguments = json.loads(tool_call.function.arguments)
        function_output = function_to_call(ticker =function_arguments['ticker'], periodo = function_arguments['periodo'])
        mensagens.append(
            {
                'tool_call_id': tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_output
            }
        )

In [7]:
mensagens

[{'role': 'user',
  'content': 'Qual é a cotação da ambev atual e qual é sua variação de preço em 1 ano?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_P9oZt26IwxMx6QMVgw5NsSLe', function=Function(arguments='{"ticker": "ABEV3.SA", "periodo": "1d"}', name='retorna_cotacao_historica'), type='function'), ChatCompletionMessageToolCall(id='call_BpNGG0EzJ9YiS6wnJtINHz6R', function=Function(arguments='{"ticker": "ABEV3.SA", "periodo": "1y"}', name='retorna_cotacao_historica'), type='function')]),
 {'tool_call_id': 'call_P9oZt26IwxMx6QMVgw5NsSLe',
  'role': 'tool',
  'name': 'retorna_cotacao_historica',
  'content': '{"01-27-2025":11.0100002289}'},
 {'tool_call_id': 'call_BpNGG0EzJ9YiS6wnJtINHz6R',
  'role': 'tool',
  'name': 'retorna_cotacao_historica',
  'content': '{"01-30-2024":13.0399999619,"02-09-2024":12.9499998093,"02-23-2024":12.8900003433,"03-06-2024":12.6899995804,"03-18-2024"

##### 5º) Fazendo a pergunta final


In [8]:
resp_final = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=mensagens
)

In [10]:
resp_final.choices[0].message.content

'A cotação atual da Ambev é de R$ 11,01. \n\nEm um período de 1 ano, a cotação da Ambev variou de R$ 13,04 para R$ 11,01, o que representa uma queda de aproximadamente 15,58%.'

##### 6º) Criando um chatBOT de finanças


In [12]:
def chat(modelo = 'gpt-3.5-turbo-0125', maximo_tokens = 4000, temperatura = 0):

    print('Bem vindo ao chatbot financeiro Otaviofinance')
    
    


    
    registro_conversa = []
    continua = True
    
    while continua:
        entrada = input('User: ')
        print('\n\nUser: ' + entrada)
        entrada_adaptada = {'role':'user', 'content': entrada}
        registro_conversa.append(entrada_adaptada)
        

        resposta = client.chat.completions.create(
        model = 'gpt-3.5-turbo-0125',
        messages = registro_conversa,
        tools = tools,
        tool_choice = 'auto')

        mensagem_resp = resposta.choices[0].message # isso representa a mensagem que o chat deu, leia em resposta
        tool_calls = mensagem_resp.tool_calls

        if tool_calls:
            registro_conversa.append(mensagem_resp)
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = funcoes_disponiveis[function_name]
                function_arguments = json.loads(tool_call.function.arguments)
                function_output = function_to_call(ticker =function_arguments['ticker'], periodo = function_arguments['periodo'])
                registro_conversa.append(
                    {
                        'tool_call_id': tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_output
                    }
                )
            
            
            Rfinal = ''
            resposta = client.chat.completions.create(
            messages = registro_conversa,
            model=modelo,
            max_tokens=maximo_tokens,
            temperature=temperatura,
            stream = True
            )
            
            print('\nAssistant: ', end='')
            for chunks in resposta:
                pedaco_de_texto = chunks.choices[0].delta.content
                if pedaco_de_texto != None: ## o último content tem esse "none" não sei porque
                    Rfinal += pedaco_de_texto
                    print(pedaco_de_texto, end = '')
            registro_conversa.append({'role':'assistant', 'content': Rfinal})
            continua = input('Deseja Continuar? (sim ou não)')
            if continua == 'sim':
                continua = True
            else:
                continua = False

        else:
            Rfinal = ''
            resposta = client.chat.completions.create(
            messages = registro_conversa,
            model=modelo,
            max_tokens=maximo_tokens,
            temperature=temperatura,
            stream = True
            )
            
            print('\nAssistant: ', end='')
            for chunks in resposta:
                pedaco_de_texto = chunks.choices[0].delta.content
                if pedaco_de_texto != None: ## o último content tem esse "none" não sei porque
                    Rfinal += pedaco_de_texto
                    print(pedaco_de_texto, end = '')
            registro_conversa.append({'role':'assistant', 'content': Rfinal})
            continua = input('Deseja Continuar? (sim ou não)')
            if continua == 'sim':
                continua = True
            else:
                continua = False
    return registro_conversa

In [13]:
chat1 = chat()

Bem vindo ao chatbot financeiro Otaviofinance


User: Qual é a cotação da ambev atual e qual é sua variação de preço em 1 ano?

Assistant: A cotação atual da Ambev (ABEV3.SA) é de R$ 11,00. 

Em um período de 1 ano, a variação de preço da Ambev foi a seguinte:
- Há 1 ano: R$ 13,04
- Hoje: R$ 11,00

Portanto, a variação de preço em 1 ano foi de -15,61%.

User: agora me diga o mesmo para a ação da apple

Assistant: A cotação atual da Apple (AAPL) é de $231,60.

Em um período de 1 ano, a variação de preço da Apple foi a seguinte:
- Há 1 ano: $188,04
- Hoje: $231,60

Portanto, a variação de preço em 1 ano foi de 23,17%.